In [ ]:
from train.training_session_v0 import load_training_session
import multiprocessing
import torch

device='cuda'

session = load_training_session(
    save_path='saves/Dec17_MSE_baseline_temp_grad/epoch_5',
    studies={"gwilliams2023": 'audio'},
    data_path='data',
)

dataloader = session.get_dataloader(buffer_size=1, num_workers=1, max_cache_size=100)

recording = session.recordings[0]

print(
    f'Showing recording: {recording.study_name}_{recording.subject_id}_{recording.task_id}'
)

dataloader.start_fetching(
    recordings=[
        session.recordings[0]
    ]
)
batch = dataloader.get_recording()
brain, audio, recording = batch.brain_segments['all'].to(device), batch.audio_segments.to(device), batch.recording

conditions = {
    "study": f'{recording.study_name}',
    "subject": f'{recording.study_name}_{recording.subject_id}',
}
session.model.to(device)

with torch.no_grad():
    pred, _ = session.model(
        x=brain, 
        recording=recording, 
        conditions=conditions , 
        train=False
    )
    
dataloader.stop()

In [ ]:
from utils.plot import mel_spectrogram

mel_spectrogram(
    x=audio[:5].to('cpu'),
    max_plots=5,
    x_pred=pred[:5].to('cpu')
)